In [20]:
import os
import sys
import io
import json
from tqdm import tqdm
from PIL import Image, ImageChops
import time
import pickle
from google.cloud import vision

client = vision.ImageAnnotatorClient()


In [42]:
with open("new_processed1.json","r") as file:
    data = json.load(file)

In [43]:
c = 0
for k,v in data.items():
    if(v == -1):
        c+=1
print(c,len(list(data.keys())))

0 10262


In [15]:
##### GET LABELS IF NOT PROCESSED HASH #####
##### GET LABELS IF NOT PROCESSED HASH #####
##### GET LABELS IF NOT PROCESSED HASH #####
def start(hashes,client):
    x = 1
    not_done = 0

    for ihash,i_url in tqdm(hashes.items()):

        if(ihash in processed and processed[ihash] == -1):
            continue
        elif(ihash in processed and 'filenames' in processed[ihash] and i_url not in processed[ihash]['filenames']):
            processed[ihash]['count'] += 1
            processed[ihash]['filenames'][i_url] = 1
        elif(ihash in processed and 'filenames' in processed[ihash] and i_url in processed[ihash]['filenames']):
            continue
        else:
            try:
                temp = label(i_url,client)
                processed[ihash] = temp
                if('count' not in processed[ihash]):
                    processed[ihash]['count'] = 1
                if('filename' not in processed[ihash]):
                    processed[ihash]['filenames'] = {}
                    processed[ihash]['filenames'][i_url] = 1
            except Exception as e:
                print(str(e))
                processed[ihash] = -1

        if(x == 5000):
            not_done = 1
            break
        x += 1



    with open("new_processed1.json","w") as file:
        json.dump(processed,file)

    return not_done

In [16]:
def label(path,client):


    file_name = os.path.abspath(path)


    with io.open(file_name, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.label_detection(image=image)
    labels = response.label_annotations
    answer = {}
    answer['labels'] = []

    for item in labels:
        answer['labels'].append((item.description,item.score))

    return answer

In [17]:
a = 1
while(a == 1):
    
    path = 'final' #change to final
    data = []
    processed = {}
    

    ##### AGGREGATE ALL FINAL WEEKS #####
    ##### AGGREGATE ALL FINAL WEEKS #####
    ##### AGGREGATE ALL FINAL WEEKS #####

    for d in os.listdir('../runs/'):
        if(path in d):
            with open('../runs/'+d,'r') as file:
                    t_data = json.load(file)
            data = data + t_data

    if(os.path.exists("new_processed1.json")):
        with open("new_processed1.json","r") as file:
            processed = json.load(file)

    hashes = {}
    for item in data:
        if(item['img_hash'] not in processed):
            hashes[item['img_hash']] = '../'+item['img_url']

    a = start(hashes,client)
    time.sleep(60)

100%|██████████| 262/262 [01:13<00:00,  3.57it/s]


In [14]:
label("rooney.png",client)

{'labels': [('Forehead', 0.9846563935279846),
  ('Sports uniform', 0.97693932056427),
  ('Smile', 0.9732518792152405),
  ('Sports jersey', 0.931155264377594),
  ('Jersey', 0.9185339212417603),
  ('Sleeve', 0.8724493980407715),
  ('Player', 0.8613125681877136),
  ('Gesture', 0.852604866027832),
  ('Collar', 0.8311039805412292),
  ('Sportswear', 0.8274717926979065)]}

# Adjust Counter and Filenames

In [26]:
temp_data = []
for d in os.listdir('../runs/'):
    with open('../runs/'+d,'r') as file:
        t_data = json.load(file)
    temp_data = temp_data + t_data

In [32]:
len(temp_data)

937652

In [45]:
data['8000000000000000']

{'labels': [['Circle', 0.6910735964775085],
  ['Event', 0.6894243359565735],
  ['Font', 0.6847700476646423],
  ['Electric blue', 0.6739935278892517],
  ['Pattern', 0.6677177548408508],
  ['Twig', 0.6632195115089417],
  ['Eyewear', 0.634919285774231],
  ['Rectangle', 0.6347272396087646],
  ['Peach', 0.6189606785774231],
  ['Logo', 0.611185610294342]],
 'count': 139648,
 'filenames': {'../static/run3_2020-05-30_control_NoIntent_94_6.png': 1,
  '../static/run7_2020-06-11_adult_NoIntent_85_RTB_image_42.png': 1,
  '../static/run7_2020-06-11_adult_NoIntent_85_RTB_image_39.png': 1,
  '../static/run7_2020-06-11_adult_NoIntent_85_RTB_image_38.png': 1,
  '../static/run7_2020-06-11_adult_NoIntent_85_RTB_image_43.png': 1,
  '../static/run7_2020-06-11_adult_NoIntent_85_RTB_image_23.png': 1,
  '../static/run7_2020-06-11_adult_NoIntent_85_RTB_image_25.png': 1,
  '../static/run7_2020-06-11_adult_NoIntent_85_RTB_image_22.png': 1,
  '../static/run7_2020-06-11_adult_NoIntent_85_RTB_image_19.png': 1,
  '.

In [34]:
temp_data[0]

{'img_hash': '0000000000000000',
 'RTB': 1,
 'img_url': 'static/run7_2020-06-11_adult_NoIntent_85_RTB_image_40.png',
 'filename': 'run7_2020-06-11_adult_NoIntent_85_RTB_image_40.png'}

In [44]:
for item in temp_data:
    chash = item['img_hash']
    imgurl= item['img_url']
    
    data[chash]['filenames']['../'+imgurl] = 1
    data[chash]['count'] += 1
    
with open("new_processed2.json","w") as file:
    json.dump(data,file)